# **Анализ: какие факторы влияют на погашение кредита в срок**

## Цель:
найти зависимость платежеспособности от другой информации о клиентах

## План работы
1. Подготовка данных для работы
  - импорт библиотеки;
  - инструкция по загрузке таблицы;
  - загрузка таблицы.
2. Предобработка данных
  - проверка типов данных, выявление пропусков;
  - исправление очевидных ошибок в данных;
  - проверка содержания данных и его исправление;
  - удаление полных дубликатов;
  - категоризация данных.
3. Выявление зависимостей
  - зависимость задолженностей по кредиту от количества детей;
  - зависимость задолженностей по кредиту от семейного положения;
  - зависимость задолженностей по кредиту от уровня дохода;
    - зависимость уровня дохода от друих факторов;
  - зависимость задолженностей по кредиту от целей кредита;
4. Общий вывод

### 1. Подготовка данных для работы

In [ ]:
#импортировать библиотеку для работу
import pandas as pd

In [ ]:
#загрузить таблицу
data = pd.read_csv('data.csv')

**Описание данных**

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### 2. Предобработка даных

####2.1. Изучаю таблицу

Ообращаю внимание на названия столбцов, хранящиеся в них значения, проверяю, есть ли среди них нулевые и в каком количестве, а также тип данных в столбцах

In [ ]:
#вывести первые 10 строк полной таблицы
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
#получить информацию об общем количестве строк (RangeIndex), столбцов (Data columns), их названиях (Column),
#ненулевых (Non-Null Count) и пропущенных (RangeIndex - Non-Null Count) значениях, а также типе данных (Dtype)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
#узнаем, сколько данных пропущено в столбцах total_income и в days_employed (в процентном соотношении)
print('Пропущено данных в days_employed: ', round(data['days_employed'].isnull().sum()/len(data)*100,0), '%')
print('Пропущено данных в total_income: ', round(data['total_income'].isnull().sum()/len(data)*100,0), '%')

Пропущено данных в days_employed:  10.0 %
Пропущено данных в total_income:  10.0 %


**Вывод:** 
1. В столбце days_employed есть отрицательные значения. Нужно уточнить у операторов, как стаж работы мог оказаться отрицательным и исправить ошибку. У меня нет возможности узнать причину, поэтому выдвигаю гипотезу: ошибка возникла при переносе данных, все отрицательные значения нужно заменить на положительные (убрать знак "-").
2. Некоторые числовые данные хранятся в формате float (вещественный). Вычисления удобнее проводить с форматом integer (целочисленный), поэтому формат нужно изменить.
3. В столбцах days_employed и total_income есть нулевые значения — 10% от общего количества данных в каждом столбце.
Можно уточнить у операторов, в чем причина пропусков. У меня нет возможности узнать причину, поэтому выдвигаю гипотезу: нулевые значения не ошибочны, а взаимосвязаны. Возможно, заемщик никогда не работал, а значит, не получал зарплату.
В любом случае, нулевые значения будут мешать при анализе, поэтому нужно их заменить.

---

####2.2. Исправляю ошибки в данных

- меняю отрицательное значение days_employed на положительное;
- избавляюсь от нулевых значений: в total_income использую медианное значение, а в days_employed — среднее*;
- изменяю тип данных float на integer;
- снова проверяю данные.
---
*Медианное значение коректнее использовать при работе с данными о финансах, т.к. оно показывает реальную картину распределения дохода. 
Среднее значение можно использовать при работе с данными об опыте работы, т.к. здесь мы не обращаем внимание на выбросы — слишком большие или слишком маленькие значения

In [ ]:
#исправить отрицательные значения в days_employed на положительные
def to_positive (row):
  if row['days_employed']<0:
    return row['days_employed']*(-1)
  else:
    return row['days_employed']

data['days_employed'] = data.apply(to_positive, axis=1)

In [ ]:
#заменить нулевое значение в days_employed на среднее
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())

#заменить нулевое значение в total_income на медианное
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

In [ ]:
#сменить тип данных с float на integer, чтобы удобнее проводить расчеты
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [ ]:
#получить информацию об исправленных данных: типе данных и нулевых значениях
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод:**
  1. Нулевые значения заменила на средние (days_employed) и медианные (total_income) — теперь все значения ненулевые.
  2. Исправила отрицательные значения в столбце days_employed.
  3. Привела численные данные к формату integer, чтобы использовать их в вычислениях.

---

####2.3. Проверю, нет ли ошибок в других данных

Например, в распределении по полу или семейному положению.

In [ ]:
#проверить нечисленные значения
print(
data['education'].value_counts(),
data['family_status'].value_counts(),
data['gender'].value_counts(),
data['income_type'].value_counts(),
data['purpose'].value_counts(),
sep='\n\n'
)


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   79

В столбце sex есть ошибка — "xna" вместо пола. Только одна запись с ошибкой, поэтому ее можно удалить — данные от этого не пострадают.

In [ ]:
#проверить целочисленные значения
print(
data['children'].value_counts(),
data['dob_years'].value_counts(),
data['debt'].value_counts(),
sep='\n\n'
)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64


- в столбце children есть ошибки в данных: у 76 клиентов 20 детей (маловероятно), у 47 — отрицательное количество (невероятно). Эти данные нужно исправить. Раз нет возможности уточнить информацию, предположу, что -1 на самом деле 1, а вместо 20 детей запишу медианное значение;
- в столбце dob_years видим 101 клиента с нулевым возрастом. Скорее всего, они не указали дату рождения или ее не перенесли при заполнении данных. Нужно обратиться к операторам и запросить данные. Такой возможности у меня нет, поэтому избавимся от 101 "новорожденного", эта потеря незначительна для имеющегося количества данных.

In [ ]:
#убрать запись с 'xna' в столбце sex
data = data.loc[data['gender']!='xna']

#убрать записи с '0' в столбце dob_years
data = data.loc[data['dob_years']!=0]

#исправить отрицательное значение на положительное и маловероятное (20) количество детей на медианное в столбце children
data.loc[data['children']==-1, 'children']=1
data.loc[data['children']==20, 'children']=data['children'].median()

**Вывод:**
  1. Даже в ненулевых данных могут содержаться ошибки, поэтому нужно их проверять.
  2. От потери небольшого количества ошибочных данных качество анализа не пострадает, поэтому я убрала записи с ошибками в столбцах sex и dob_years. Возраст можно было заменить медианным значением (реально показывающим, какой усредненный возраст клиентов), если бы данных было значительно больше сотни.
  3. В столбце purpose некоторые данные совпадают по смыслу, но записаны по-разному. Можно сократить их количество, объединив эти значения по категориям.
  4. Я не проверяла значения в столбце total_income, потому что их разброс еще больше, чем в столбце dob_years. Однако эти значения понадобятся при анализе данных, поэтому я обращусь к ним позже.

####2.4. Ищем дубликаты

Если записи дублируют друг друга, от них можно избавиться и сократить количество данных для анализа. Для этого нужно привести данные с типом object к одному регистру, затем проверить строки на дубликаты, а повторы убрать.

In [ ]:
#перевести данные типа object в нижний регистр — всего 5 столбцов
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

In [ ]:
#посмотреть количество дубликатов
data.duplicated().sum()

71

In [ ]:
#убрать полные дубликаты и перезаписать индекс
data = data.drop_duplicates().reset_index(drop=True)

**Вывод:** теперь данных меньше и они полностью уникальные, повторы не повлияют на поиск зависимостей и не исказят выводы.



---



####2.5. Категоризируем данные

  - в столбце purpose сходные по смыслу категории записаны по-разному, их можно объединить. Анализировать так проще. Для этого нужно лемматизировать данные в столбце purpose (привести каждое слово фразы к начальной форме), а потом определить ключевые слова;
  - в столбце total_income слишком большой разброс данных. Чтобы использовать их для анализа, нужно категоризировать доход, выделив несколько общих групп (воспользуюсь квартилями и медианой).

In [ ]:
#импортировать библиотеку и инструмент для лемматизации
import pymystem3
from pymystem3 import Mystem

In [ ]:
#записать все уникальные значения из столбца purpose в переменную для лемматизации — data_for_lemmatizer
#каждое уникальное значение повторяется в столбце несколько сотен раз
#проверить, сколько раз значения появляются в данных, можно выше, на шаге с поиском ошибок в данных типа object
data_for_lemmatizer = data['purpose'].unique()
print(data_for_lemmatizer)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [ ]:
#лемматизировать данные из data_for_lemmatizer — получить набор слов в начальной форме для каждой фразы
#сохранить полученные наборы в переменную lemmatized_purpose 
lemmatized_purpose = []
for purpose in data_for_lemmatizer:
  lemmatized_purpose.append(list(Mystem().lemmatize(purpose)))
print(lemmatized_purpose)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['жилье', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'], ['автомобиль', '\n'], ['заниматьс

In [ ]:
#заменить оригинальную запись в столбце purpose соответствующей ей лемматизированной из lemmatized_purpose
for l in range (len(data_for_lemmatizer)):
  for i in range (len(data['purpose'])):
    if data_for_lemmatizer[l]==data['purpose'][i]:
      data['purpose'][i]=str(lemmatized_purpose[l])
    else:
      continue

<ipython-input-19-5be1fb55a41a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['purpose'][i]=str(lemmatized_purpose[l])


Я работаю с новыми переменными, а не применяю лемматизацию сразу к таблице, чтобы сэкономить время. В Google Colab операция со столбцом длится около получаса, а лемматизация уникальных данных и замена соответствующих им в таблице — всего несколько минут

In [ ]:
#переменная lemmatized_purpose была списком, и хранились в ней тоже списки
#чтобы продолжить работу, нужно распаковать данные в один список
lemmatized_purpose = [word for purpose in lemmatized_purpose for word in purpose]
print(lemmatized_purpose)

['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'покупка', ' ', 'недвижимость', '\n', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'недвижимость', '\n', 'строительство', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'жилье', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'сделка', ' '

In [ ]:
#посчитать, с какой частотой встречаются леммы из уникальных данных столбца purpose
from collections import Counter
print(Counter(lemmatized_purpose).most_common())

[(' ', 59), ('\n', 38), ('покупка', 10), ('недвижимость', 10), ('автомобиль', 9), ('образование', 9), ('жилье', 7), ('с', 5), ('операция', 4), ('на', 4), ('свой', 4), ('свадьба', 3), ('строительство', 3), ('получение', 3), ('высокий', 3), ('дополнительный', 2), ('для', 2), ('коммерческий', 2), ('жилой', 2), ('заниматься', 2), ('сделка', 2), ('приобретение', 1), ('сыграть', 1), ('проведение', 1), ('семья', 1), ('собственный', 1), ('подержать', 1), ('со', 1), ('подержанный', 1), ('профильный', 1), ('сдача', 1), ('ремонт', 1)]


Промежуточный результат:

- часто встречаются слова "покупка", "недвижимость", "автомобиль", "образование" и "жилье";
- "покупка" может относиться и к недвижимости, и к машине, поэтому ее из списка ключевых слов я убираю;
- вместо покупки вношу слово "свадьба": пусть в наборе уникальных данных было всего три разных варианта кредита на свадьбу, зато при подсчете общих данных по столбцу purpose клиентов очень интересовала эта категория.

In [ ]:
#категоризировать цели кредита
#каждому значению столбца purpose присвоить категорию, соответствующую входящим в запись леммам
#н-р: если цель была "покупка жилье" ("покупка", "жилье" после лемматизации), то запись попадет в категорию "недвижимость"
def replace_purpose (purp):
  if 'недвижимость' in purp:
    return 'недвижимость'
  if 'автомобиль' in purp:
    return 'авто'
  if 'образование' in purp:
    return 'образование'
  if 'жилье' in purp:
    return 'недвижимость'
  if 'свадьба' in purp:
    return 'свадьба'
#применить функцию
data['purpose_id'] = data['purpose'].apply(replace_purpose)

In [ ]:
#категоризировать доход
#каждому значению столбца total_income присвоить категорию в зависимости от диапазона, в который попадает запись
def replace_income (total_income):
  if total_income <= data['total_income'].quantile(0.25):
    return 'низкий'
  if total_income > data['total_income'].quantile(0.25) and total_income <= data['total_income'].median():
    return 'средний'
  if total_income > data['total_income'].median() and total_income < data['total_income'].quantile(0.75):
    return 'выше среднего'
  if total_income >= data['total_income'].quantile(0.75):
    return 'высокий'
#применить функцию
data['total_income_id'] = data['total_income'].apply(replace_income)

In [ ]:
#вывести первые 10 строк итоговой таблицы
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,total_income_id
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,"['покупка', ' ', 'жилье', '\n']",недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,"['приобретение', ' ', 'автомобиль', '\n']",авто,средний
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,"['покупка', ' ', 'жилье', '\n']",недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,"['дополнительный', ' ', 'образование', '\n']",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,"['сыграть', ' ', 'свадьба', '\n']",свадьба,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,"['покупка', ' ', 'жилье', '\n']",недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,"['операция', ' ', 'с', ' ', 'жилье', '\n']",недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,"['образование', '\n']",образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,"['на', ' ', 'проведение', ' ', 'свадьба', '\n']",свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,"['покупка', ' ', 'жилье', ' ', 'для', ' ', 'се...",недвижимость,средний


####Вывод:
1. Чтобы категоризировать цели кредита, я лемматизировала данные и определила цель каждого клиента по ключевым словам в записи. Результат хранится в новом столбце purpose_id таблицы data.
2. Чтобы категоризировать доход, я использовала значения квартилей. Категории хранятся в новом столбце total_income_id таблицы data.

- Низкий доход — все данные меньше и равные числу, определяющему Q1 (>25% данных).
- Средний доход - все данные больше Q1, но меньше медианного значения Q2 (50%) или равные ему.
- Выше среднего — все данные больше медианного значения, но меньше Q3 (>75% данных).
- Выскоий доход — все данные больше или равные Q3.

3. Категоризация облегчит анализ данных, для составления которого в финале потребуется сводная таблица, в т.ч. с информацией о доходе клиента и целях кредита.

### 3. Анализ зависимостей

####3.1. Зависимость задолженностей по кредиту от наличия детей.

In [ ]:
data.groupby('children')[['debt']].mean()*100

,debt
children,
0,7.561893
1,9.134119
2,9.514468
3,8.231707
4,9.756098
5,0.000000


**Вывод:**
- чем больше среднее значение (я умножила его на 100, чтобы сразу получить данные в процентах), тем лучше;
- хуже всего ситуация с задолженностью по кредиту у бездетных клиентов;
- самый низкий показатель задолженности у клиентов с 2 детьми;
- при этом у клиентов с 5 детьми вообще нет задолженности. Возможно, эти клиенты (напомню, их всего 9 человек) ранее просто не брали кредиты.



---



####3.2. Зависимость задолженностей по кредиту от семейного положения.

In [ ]:
data.groupby('family_status')[['debt']].mean()*100

,debt
family_status,
в разводе,7.172996
вдовец / вдова,6.498952
гражданский брак,9.346247
женат / замужем,7.542718
не женат / не замужем,9.770938


**Вывод:**
- меньше всего задолженностей по кредиту у не женатых клиентов;
- немногим хуже результат у клиентов в гражданском браке;
- больше всего задолженностей по кредиту у вдов/вдовцов.



---



####3.3. Зависимость задолженностей по кредиту от уровня дохода.

In [ ]:
data.groupby('total_income_id')[['debt']].mean()*100

,debt
total_income_id,
высокий,7.154898
выше среднего,8.881119
низкий,7.960292
средний,8.535630


**Вывод:**
- больше всего задолженностей по кредиту у клиентов с высоким уровнем дохода — 7,1%;
- меньше всего задолженностей по кредиту у клиентов с доходом выше среднего уровня — 8,8%.

#####3.3.1. Какие факторы преобладают у клиентов с высоким уровнем дохода

Проверю, почему клиенты с высоким уровнем дохода чаще среди должников. Возможно, это связано с количеством детей или с семейным положением.

In [ ]:
data.pivot_table(index = 'total_income_id', columns = 'children', values = 'debt', aggfunc='count')

children,0,1,2,3,4,5
total_income_id,,,,,,
высокий,3452,1258,527,91,7,4
выше среднего,2819,1001,388,68,13,1
низкий,3636,1109,519,68,5,2
средний,4190,1471,605,101,16,2


Итак, у людей с высоким уровнем дохода обычно нет детей. У бездетных клиентов ситуация с выплатой кредитов действительно хуже.

In [ ]:
data.pivot_table(index = 'total_income_id', columns = 'family_status', values = 'debt', aggfunc='count')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
total_income_id,,,,,
высокий,304,167,1039,3128,701
выше среднего,247,170,819,2507,547
низкий,292,315,1037,3033,662
средний,342,302,1235,3622,884


Как уже известно, овдовевшие клиенты и клиенты в разводе обычно хуже платят по кредиту. Но среди клиентов с высоким уровнем дохода больше замужних и женатых, а также состоящих в гражданском браке. Клиенты в гражданском браке — одна из самых надежных категорий.

In [ ]:
data.pivot_table(index = 'total_income_id', columns = 'purpose_id', values = 'debt', aggfunc='count')

purpose_id,авто,недвижимость,образование,свадьба
total_income_id,,,,
высокий,1086,2742,937,574
выше среднего,823,2198,811,458
низкий,1039,2666,1036,598
средний,1336,3158,1211,680


Клиенты с высоким уровнем дохода чаще берут кредит на недвижимость. В следующем шаге я увижу, что в этой категории наблюдается больший уровень задолженности.

Предположу, что категория "уровень дохода" как фактор должна рассматриваться только вместе с другими категориями, лишь на ее основании нельзя делать вывод о погашении кредита в срок.



---



####3.4. Зависимость задолженностей по кредиту от цели

In [ ]:
data.groupby('purpose_id')[['debt']].mean()*100

,debt
purpose_id,
авто,9.337068
недвижимость,7.237087
образование,9.261577
свадьба,7.965368


**Вывод:**
- больше всего задолженностей по кредиту у клиентов, желающих взять кредит на недвижимость — 7,2%;
- меньше всего задолженностей по кредиту у клиентов, желающих взять кредит на авто — 9,3%.



---



####Вывод:
1. Идеальный клиент — не женат или в гражданском браке, с 2 детьми, с уровнем дохода выше среднего, желающий взять кредит на авто.
2. Худший клиент — вдовец без детей, с высоким уровнем дохода, желающий взять кредит на недвижимость.

## Итог:
1. В данных много неточностей, поэтому стоит обратить внимание на форму заполнения информации или процесс сбора данных. Возможно, этих неточностей (которые могут привести к ошибкам) можно избежать.

2. Чаще всего люди берут кредит на недвижимость, авто, свадьбу и образование.

3. Выплатит ли клиент кредит в срок, зависит от нескольких, возможно, неочевидных факторов.

- количество детей — вероятно, наличие детей делает человека более ответственным и в вопросах финансов;
- семейное положение — наличие семьи не гарантирует выплату кредита, но потеря партнера (развод или вдовство) снижают шанс на погашение кредита в срок;
- уровень дохода — ожидаемо, у людей с низким уровнем дохода выплаты по кредиту хуже, но и люди с высоким уровнем дохода часто среди должников.* Клиенты с доходом выше среднего чаще выплачивают кредит в срок.

4. Указанные факторы рекомендую рассматривать в совокупности, не делать вывод на основании лишь количества детей или уровня дохода: 
  
  *Я выяснила, что среди клиентов с высоким уровнем дохода много бездетных, а также желающих взять кредит на недвижимость. В этих категориях как раз наблюдается частая задолженность по кредиту. Предполагаю, что категория "уровень дохода" как фактор должна рассматриваться вместе с другими категориями.